In [1]:
import pandas as pd
import numpy as np


In [2]:
df_final=pd.read_csv(r"C:\Users\Ramakrushna das\OneDrive\Desktop\Twitter_Data.csv")
df_final.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [3]:
df_final.isnull().sum()

clean_text    4
category      7
dtype: int64

In [4]:
df_final.dropna(subset=['clean_text','category'],axis=0,inplace=True)

In [5]:
df_final.isnull().sum()

clean_text    0
category      0
dtype: int64

In [6]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [7]:
import re 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Ramakrushna
[nltk_data]     das\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
df_final=df_final.reset_index(drop=True)

In [9]:
ps=PorterStemmer()
corpus=[]
for i in range(0, len(df_final)):
    review = re.sub('[^a-zA-Z]', ' ', df_final['clean_text'][i])
    review = review.lower()
    review = review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
corpus

['modi promis minimum govern maximum govern expect begin difficult job reform state take year get justic state busi exit psu templ',
 'talk nonsens continu drama vote modi',
 'say vote modi welcom bjp told rahul main campaign modi think modi relax',
 'ask support prefix chowkidar name modi great servic confus read crustal clear crass filthi nonsens see abus come chowkidar',
 'answer among power world leader today trump putin modi may',
 'kiya tho refresh maarkefir comment karo',
 'surat women perform yagna seek divin grace narendra modi becom',
 'come cabinet scholar like modi smriti hema time introspect',
 'upcom elect india saga go import pair look current modi lead govt elect deal brexit combin weekli look juici bear imho',
 'gandhi gay modi',
 'thing like demonetis gst good servic tax upper cast would sort either view favour say need give time cast like dalit muslim modi constitu',
 'hope tuthukudi peopl would prefer honest well behav nationalist courag likli minist modi cabinet vo

In [11]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Ramakrushna
[nltk_data]     das\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [14]:
words

[['modi',
  'promis',
  'minimum',
  'govern',
  'maximum',
  'govern',
  'expect',
  'begin',
  'difficult',
  'job',
  'reform',
  'state',
  'take',
  'year',
  'get',
  'justic',
  'state',
  'busi',
  'exit',
  'psu',
  'templ'],
 ['talk', 'nonsens', 'continu', 'drama', 'vote', 'modi'],
 ['say',
  'vote',
  'modi',
  'welcom',
  'bjp',
  'told',
  'rahul',
  'main',
  'campaign',
  'modi',
  'think',
  'modi',
  'relax'],
 ['ask',
  'support',
  'prefix',
  'chowkidar',
  'name',
  'modi',
  'great',
  'servic',
  'confus',
  'read',
  'crustal',
  'clear',
  'crass',
  'filthi',
  'nonsens',
  'see',
  'abus',
  'come',
  'chowkidar'],
 ['answer',
  'among',
  'power',
  'world',
  'leader',
  'today',
  'trump',
  'putin',
  'modi',
  'may'],
 ['kiya', 'tho', 'refresh', 'maarkefir', 'comment', 'karo'],
 ['surat',
  'women',
  'perform',
  'yagna',
  'seek',
  'divin',
  'grace',
  'narendra',
  'modi',
  'becom'],
 ['come',
  'cabinet',
  'scholar',
  'like',
  'modi',
  'smriti

In [15]:
import gensim
model=gensim.models.Word2Vec(words,window=10,min_count=2)

In [16]:
def avg_word_2_vec(doc, model, vector_size):
    word_vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size) 

In [17]:
from tqdm import tqdm

In [18]:
X = []
for i in tqdm(range(len(words))):
    X.append(avg_word_2_vec(words[i], model, vector_size=model.vector_size))

100%|██████████| 162893/162893 [02:47<00:00, 970.49it/s] 


In [19]:
len(X)

162893

In [20]:
X_new=np.array(X)

In [21]:
df = pd.DataFrame(X_new)
print(df.head())

         0         1         2         3         4         5         6   \
0 -0.181918  0.096272  0.049222 -0.202388  0.412637 -0.463952  0.128372   
1  0.199922 -0.112419  0.378429  0.260431  0.171148 -0.188945  0.294467   
2  0.821168 -0.242522 -0.139773  0.394307  0.189968 -0.059921  0.491213   
3  0.425077  0.081163 -0.037985  0.317092  0.524222  0.079072  0.156925   
4 -0.097190 -0.019547  0.571217 -0.132473  0.748115  0.094766  0.115496   

         7         8         9   ...        90        91        92        93  \
0  0.443881  0.366759 -0.755121  ...  0.604113  0.840833 -0.790670  0.152781   
1  0.534505  0.385740 -0.445011  ...  0.929926 -0.409671  0.121879  0.199551   
2  0.469894  0.259579 -0.259966  ...  0.447422  0.246626 -0.750619  0.582925   
3  0.735236  0.361327 -0.806389  ...  0.450374 -0.128288  0.131031 -0.351495   
4  0.375405  0.976195 -0.308286  ...  0.526692 -0.205155  0.525436 -0.034856   

         94        95        96        97        98        99  
0  0

In [22]:
len(df_final['category'])

162969

In [23]:
missing_values = [[i, j, k] for i, j, k in zip(list(map(len, corpus)), corpus, df_final['clean_text']) if i < 1]
print(f"Number of missing values: {len(missing_values)}")


Number of missing values: 76


In [24]:
filtered_df_final = df_final[list(map(lambda x: len(x) > 0, corpus))]
y = filtered_df_final['category'].values  
print(len(filtered_df_final), len(X))

162893 162893


In [25]:
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.181918,0.096272,0.049222,-0.202388,0.412637,-0.463952,0.128372,0.443881,0.366759,-0.755121,...,0.604113,0.840833,-0.790670,0.152781,0.463371,-0.320057,0.422367,-0.753327,-0.453012,0.371971
1,0.199922,-0.112419,0.378429,0.260431,0.171148,-0.188945,0.294467,0.534505,0.385740,-0.445011,...,0.929926,-0.409671,0.121879,0.199551,0.069543,-0.705049,0.976442,-0.127353,0.020251,0.434680
2,0.821168,-0.242522,-0.139773,0.394307,0.189968,-0.059921,0.491213,0.469894,0.259579,-0.259966,...,0.447422,0.246626,-0.750619,0.582925,0.276394,-0.595320,0.338883,-0.455811,-0.137126,0.510197
3,0.425077,0.081163,-0.037985,0.317092,0.524222,0.079072,0.156925,0.735236,0.361327,-0.806389,...,0.450374,-0.128288,0.131031,-0.351495,0.356340,-0.926143,0.375538,-0.088836,0.067661,0.059906
4,-0.097190,-0.019547,0.571217,-0.132473,0.748115,0.094766,0.115496,0.375405,0.976195,-0.308286,...,0.526692,-0.205155,0.525436,-0.034856,1.018659,-0.382226,-0.218444,-0.523060,-0.991780,0.595318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162888,0.376923,0.281629,0.179286,-0.278710,0.592833,-0.748717,-0.078782,0.677079,-0.302674,-1.106002,...,0.563372,0.984194,-1.550112,0.724356,0.182236,-0.756741,0.975094,-0.058437,0.240195,-0.245138
162889,-0.179117,0.245988,0.416984,-0.024607,0.220547,0.678962,-0.032972,0.435713,0.311630,-0.025771,...,0.332591,-0.201615,0.409519,0.033014,0.634821,-0.684826,0.499304,-0.410568,-0.253438,-0.307656
162890,0.393403,0.005560,0.151809,0.244270,0.595587,-0.435788,0.062118,0.852881,0.218200,-0.291182,...,-0.124265,0.274720,-0.248737,0.118470,0.596458,-0.552880,0.113953,0.268398,0.185424,0.087675
162891,-1.139011,0.061448,-0.044688,-0.168499,1.014613,0.141978,-0.161280,0.537611,0.764471,-0.391089,...,0.864001,1.657315,-0.088255,0.135233,0.589940,-0.079212,-0.162150,-0.337263,0.301244,1.649136


In [26]:
len(y)

162893

In [27]:
df.shape

(162893, 100)

In [28]:
df.head

<bound method NDFrame.head of               0         1         2         3         4         5         6   \
0      -0.181918  0.096272  0.049222 -0.202388  0.412637 -0.463952  0.128372   
1       0.199922 -0.112419  0.378429  0.260431  0.171148 -0.188945  0.294467   
2       0.821168 -0.242522 -0.139773  0.394307  0.189968 -0.059921  0.491213   
3       0.425077  0.081163 -0.037985  0.317092  0.524222  0.079072  0.156925   
4      -0.097190 -0.019547  0.571217 -0.132473  0.748115  0.094766  0.115496   
...          ...       ...       ...       ...       ...       ...       ...   
162888  0.376923  0.281629  0.179286 -0.278710  0.592833 -0.748717 -0.078782   
162889 -0.179117  0.245988  0.416984 -0.024607  0.220547  0.678962 -0.032972   
162890  0.393403  0.005560  0.151809  0.244270  0.595587 -0.435788  0.062118   
162891 -1.139011  0.061448 -0.044688 -0.168499  1.014613  0.141978 -0.161280   
162892 -0.227384  0.702628  0.245448 -0.375947  0.505212  0.759926 -0.095529   

         

In [29]:
df['Output']=y

In [30]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.181918,0.096272,0.049222,-0.202388,0.412637,-0.463952,0.128372,0.443881,0.366759,-0.755121,...,0.840833,-0.790670,0.152781,0.463371,-0.320057,0.422367,-0.753327,-0.453012,0.371971,-1.0
1,0.199922,-0.112419,0.378429,0.260431,0.171148,-0.188945,0.294467,0.534505,0.385740,-0.445011,...,-0.409671,0.121879,0.199551,0.069543,-0.705049,0.976442,-0.127353,0.020251,0.434680,0.0
2,0.821168,-0.242522,-0.139773,0.394307,0.189968,-0.059921,0.491213,0.469894,0.259579,-0.259966,...,0.246626,-0.750619,0.582925,0.276394,-0.595320,0.338883,-0.455811,-0.137126,0.510197,1.0
3,0.425077,0.081163,-0.037985,0.317092,0.524222,0.079072,0.156925,0.735236,0.361327,-0.806389,...,-0.128288,0.131031,-0.351495,0.356340,-0.926143,0.375538,-0.088836,0.067661,0.059906,1.0
4,-0.097190,-0.019547,0.571217,-0.132473,0.748115,0.094766,0.115496,0.375405,0.976195,-0.308286,...,-0.205155,0.525436,-0.034856,1.018659,-0.382226,-0.218444,-0.523060,-0.991780,0.595318,1.0


In [31]:
df['Output'].unique()

array([-1.,  0.,  1.])

In [32]:
df.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
96        0
97        0
98        0
99        0
Output    0
Length: 101, dtype: int64

In [33]:
m=df.drop('Output',axis=1)
n=df['Output']

In [34]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(m,n,test_size=0.20)

In [35]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [36]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [37]:
y_pred=classifier.predict(X_test)

In [38]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

0.6289941373277265


In [39]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.60      0.27      0.37      7071
         0.0       0.66      0.65      0.66     10989
         1.0       0.62      0.79      0.69     14519

    accuracy                           0.63     32579
   macro avg       0.62      0.57      0.57     32579
weighted avg       0.63      0.63      0.61     32579

